In [ ]:
# pip install -r ../requirements.txt

In [ ]:
import os
import pandas as pd
# Get the current working directory
current_dir = os.getcwd()

# Construct the file path
file_path_hsi = os.path.join(current_dir, '../data/^HSI (20250221000000000 _ 20240510000000000).csv')

# Check if the file exists before reading
if not os.path.exists(file_path_hsi):
    raise FileNotFoundError(f"The file {file_path_hsi} does not exist.")

df_hsi = pd.read_csv(file_path_hsi)
print(df_hsi.head())


In [ ]:
import IPython
import sys

# Get kernel information
kernel_info = IPython.get_ipython().kernel

# Print kernel information
print(f"Kernel ID: {kernel_info.session.session}")
print(f"Kernel Session: {kernel_info.session}")
# print python version
print(f"Python Version: {sys.version}")

In [ ]:
import yfinance as yf

# Define stock tickers
ticker = "AAPL"  # Example: Apple stock

# Fetch historical market data
stock_data = yf.download(ticker, start="2024-01-01", end="2024-02-01")

# Display first few rows
print(stock_data.head())

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
# Get the current working directory
current_dir = os.getcwd()


# Check if the file exists before reading
if not os.path.exists(file_path_hsi):
    raise FileNotFoundError(f"The file {file_path_hsi} does not exist.")

df_hsi = pd.read_csv(file_path_hsi)

# Ensure required columns are present
required_columns = ['Date', 'Close', 'MCHI']
missing_columns = [col for col in required_columns if col not in df_hsi.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

# Convert to numeric and clean data
df_hsi['Close'] = pd.to_numeric(df_hsi['Close'].str.replace(',', ''), errors='coerce')
df_hsi['MCHI'] = pd.to_numeric(df_hsi['MCHI'], errors='coerce')

# Filter valid data points (keep rows where at least one of Close or MCHI is not NaN)
valid_data = df_hsi.dropna(subset=['Close', 'MCHI'], how='all')
valid_data = valid_data[valid_data['Date'] >= '2025-01-01'].tail(30)

# Ensure there are enough valid data points
if valid_data.empty:
    raise ValueError("No valid data points found after cleaning.")

# Convert date format
valid_data['Date'] = pd.to_datetime(valid_data['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Ensure date conversion was successful
if valid_data['Date'].isnull().any():
    raise ValueError("Date conversion failed for some entries.")
df_hsi = pd.read_csv(file_path_hsi)
# Convert to numeric and clean data
df_hsi['Close'] = pd.to_numeric(df_hsi['Close'].str.replace(',', ''), errors='coerce')
df_hsi['MCHI'] = pd.to_numeric(df_hsi['MCHI'], errors='coerce')

# Filter valid data points (keep rows where at least one of Close or MCHI is not NaN)
# valid_data = df_hsi.dropna(subset=['Close', 'MCHI'], how='all').tail(50)

# Convert date format
valid_data['Date'] = pd.to_datetime(valid_data['Date']).dt.strftime('%Y-%m-%d')

# Create plot with dual axes
plt.figure(figsize=(12, 6))
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot HSI Close (left axis)
ax1.plot(
    valid_data['Date'], 
    valid_data['Close'], 
    label='HSI Close', 
    color='blue', 
    marker='o',
    markersize=4,
    linestyle='-'
)

# Plot MCHI (right axis)
ax2.plot(
    valid_data['Date'], 
    valid_data['MCHI'], 
    label='MCHI', 
    color='orange', 
    marker='s',
    markersize=4,
    linestyle='--'
)

# Configure plot appearance
# Show every date on the x-axis
ax1.set_xticks(valid_data['Date'])  # Set ticks for all dates
ax1.set_xticklabels(valid_data['Date'], rotation=45, ha='right')  # Label all dates

ax1.set_ylabel('HSI Value', color='blue')
ax2.set_ylabel('MCHI Value', color='orange')
plt.title('HSI and MCHI Closing Prices (Valid Data Points Only)')

# Add combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Ensure 'Close' and 'MCHI' columns are of string type before using .str accessor
df_hsi['Close'] = df_hsi['Close'].astype(str)
df_hsi['MCHI'] = df_hsi['MCHI'].astype(str)

# Convert to numeric and clean data
df_hsi['Close'] = pd.to_numeric(df_hsi['Close'].str.replace(',', ''), errors='coerce')
df_hsi['MCHI'] = pd.to_numeric(df_hsi['MCHI'], errors='coerce')

# Filter valid data points where both Close and MCHI have values
valid_data = df_hsi.dropna(subset=['Close', 'MCHI'])

# Calculate daily returns (gain/loss)
valid_data['HSI_Return'] = valid_data['Close'].pct_change()
valid_data['MCHI_Return'] = valid_data['MCHI'].pct_change()

# Shift MCHI returns to align with next day's HSI returns
valid_data['MCHI_Return_Shifted'] = valid_data['MCHI_Return'].shift(-1)

# Drop rows with NaN values after shifting
valid_data = valid_data.dropna(subset=['HSI_Return', 'MCHI_Return_Shifted'])

# Calculate cross-correlation
cross_corr = valid_data['HSI_Return'].corr(valid_data['MCHI_Return_Shifted'])

# Calculate co-movement statistics
same_direction = (
    (valid_data['HSI_Return'] > 0) & (valid_data['MCHI_Return_Shifted'] > 0) |
    (valid_data['HSI_Return'] < 0) & (valid_data['MCHI_Return_Shifted'] < 0)
).sum()

opposite_direction = (
    (valid_data['HSI_Return'] > 0) & (valid_data['MCHI_Return_Shifted'] < 0) |
    (valid_data['HSI_Return'] < 0) & (valid_data['MCHI_Return_Shifted'] > 0)
).sum()

total_days = len(valid_data)

# Print results
print(f"Cross-Correlation (MCHI predicts HSI next day): {cross_corr:.2f}")
print(f"\nCo-Movement Statistics:")
print(f"Days MCHI and HSI moved in the same direction: {same_direction} ({same_direction/total_days:.1%})")
print(f"Days MCHI and HSI moved in opposite directions: {opposite_direction} ({opposite_direction/total_days:.1%})")
print(f"Total days analyzed: {total_days}")

# Schwab

In [ ]:
import pandas as pd

def load_and_display_csv(file_path: str, num_rows: int = 10) -> None:
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Ignore the first row
    df = df.iloc[1:]
    
    # Drop empty rows
    df = df.dropna(how='all')
    
    # Remove rows where the second column has values NaN, 'Description', 'nan', or '--'
    df = df[~df.iloc[:, 1].isin([pd.NA, 'Description', 'nan', '--'])]
    df = df.dropna(subset=[df.columns[1]])
    
    # Select the first 3 columns, the 7th column, and the last one
    selected_columns = df.iloc[:, [0, 1, 2, 6, -1]].copy()
    
    # Rename the columns
    selected_columns.columns = ['Symbol', 'Detail', 'Holding', 'Mkt Value', 'Type']
    
    # Remove '$' sign and commas, then convert 'Mkt Value' to float
    selected_columns.loc[:, 'Mkt Value'] = selected_columns['Mkt Value'].replace('[\$,]', '', regex=True).replace(',', '', regex=True).astype(float)
    
    # Convert 'Holding' to float
    selected_columns.loc[:, 'Holding'] = selected_columns['Holding'].replace(',', '', regex=True).astype(float)
    
    # Group by 'Symbol' and sum 'Holding' and 'Mkt Value'
    grouped = selected_columns.groupby('Symbol').agg({
        'Detail': 'first',
        'Holding': 'sum',
        'Mkt Value': 'sum',
        'Type': 'first'
    }).reset_index()
    
    # Round 'Holding' and 'Mkt Value' to int
    grouped['Holding'] = grouped['Holding'].round().astype(int)
    grouped['Mkt Value'] = grouped['Mkt Value'].round().astype(int)
    
    # Sort by 'Mkt Value' in descending order
    grouped = grouped.sort_values(by='Mkt Value', ascending=False)
    
    # Display the grouped columns as a Markdown table
    print(grouped.head(num_rows).to_markdown(index=False))

# Example usage
file_path = '/Users/hliu/Downloads/All-Accounts-Positions-2025-02-07-164814.csv'
load_and_display_csv(file_path, num_rows=20)

In [ ]:

load_and_display_csv(file_path, 100)

In [ ]:
from IPython.display import display, HTML

# Create an HTML block with the provided content
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SPY Stock Chart</title>
    <script>
        // Highcharts library
        !function(a){"object"==typeof module&&module.exports?module.exports=a:a(Highcharts)}(function(a){a.createElement("link",{href:"https://code.highcharts.com/css/highcharts.css",rel:"stylesheet",type:"text/css"},null,document.getElementsByTagName("head")[0]),a.createElement("script",{src:"https://code.highcharts.com/stock/highstock.js"},null,document.getElementsByTagName("head")[0]),a.createElement("script",{src:"https://code.highcharts.com/modules/exporting.js"},null,document.getElementsByTagName("head")[0])});
    </script>
</head>
<body>
    <div id="container" style="height: 600px; min-width: 100%;"></div>

    <script>
        async function fetchSPYData() {
            const response = await fetch("https://query1.finance.yahoo.com/v8/finance/chart/SPY?range=1y&interval=1d");
            const data = await response.json();
            const prices = data.chart.result[0].timestamp.map((timestamp, index) => {
                return [timestamp * 1000, data.chart.result[0].indicators.quote[0].close[index]];
            });

            Highcharts.stockChart('container', {
                rangeSelector: { selected: 1 },
                title: { text: 'SPY Stock Price' },
                series: [{
                    name: 'SPY',
                    data: prices,
                    tooltip: { valueDecimals: 2 }
                }]
            });
        }

        fetchSPYData();
    </script>
</body>
</html>
"""

# Display the HTML block
display(HTML(html_content))